In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 62kB/s 
     |████████████████████████████████| 204kB 50.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=0ca3c1ab9db0f7cc44d745b47950716f4a9a113729f0b012a19b70e705710271
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1f_9EbnywCj35EBUA32sueigxBjBJwALr'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('War and Peace by Leo Tolstoy (ru).txt')

In [ ]:
import pandas as pd

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().set("spark.ui.port", "4050")

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-12-23 10:26:46--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.90.94.177, 52.204.93.39, 52.73.16.193, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.90.94.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  38.1MB/s    in 0.3s    

2020-12-23 10:26:47 (38.1 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://27e9187b655d.ngrok.io


Подсчитай кол-во слов в документе "War and Peace by Leo Tolstoy (ru).txt"

In [ ]:
# код
from pyspark.sql import SparkSession
import pandas as pd

ss = SparkSession(sc)

In [ ]:
!pip install chardet

In [ ]:
from chardet.universaldetector import UniversalDetector
  
detector = UniversalDetector()
with open('War and Peace by Leo Tolstoy (ru).txt', 'rb') as fh:
    for line in fh:
        detector.feed(line)
        if detector.done:
            break
    detector.close()
print(detector.result)

{'encoding': 'windows-1251', 'confidence': 0.9743328310474701, 'language': 'Russian'}


In [ ]:
xx = ss.read.csv("War and Peace by Leo Tolstoy (ru).txt", sep='%s\t%s', encoding='windows-1251', header='true', inferSchema='true',)

In [ ]:
print(xx.count())
xx.columns

13201


['Война и мир. Книга 1']

In [ ]:
xx.show(5)

+--------------------+
|Война и мир. Книга 1|
+--------------------+
|Лев Николаевич То...|
|      Война и мир #1|
|В книгу вошли пер...|
|          Том первый|
|        Часть первая|
+--------------------+
only showing top 5 rows



In [ ]:
xx.filter(length(col("`Война и мир. Книга 1`")) > 0).withColumn("words", split(col("`Война и мир. Книга 1`"), "\\s+")).select(explode(col("words"))).show(40)

+------------+
|         col|
+------------+
|         Лев|
|  Николаевич|
|     Толстой|
|       Война|
|           и|
|         мир|
|          #1|
|           В|
|       книгу|
|       вошли|
|      первый|
|           и|
|      второй|
|        тома|
|      романа|
|      «Война|
|           и|
|        мир»|
|           –|
|      одного|
|          из|
|       самых|
|  знаменитых|
|произведений|
|  литературы|
|         XIX|
|       века.|
|         Том|
|      первый|
|       Часть|
|      первая|
|           I|
|        – Eh|
|       bien,|
|         mon|
|     prince.|
|       G?nes|
|          et|
|     Lucques|
|          ne|
+------------+
only showing top 40 rows



In [ ]:
xx.filter(length(col("`Война и мир. Книга 1`")) > 0).withColumn("words", split(col("`Война и мир. Книга 1`"), "\\s+")).select(explode(col("words"))).describe().show()

+-------+------------------+
|summary|               col|
+-------+------------------+
|  count|            475218|
|   mean| 320.5295652173913|
| stddev|435.44152087066476|
|    min|                  |
|    max|            № 178.|
+-------+------------------+



возможно стоило почистить текст, чтобы получить именно слова

In [ ]:
id='13yfAoONwq4rS5XrTv3IrcqcFcdgfvK9V'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1VE_9x0LQvOJpHXbXp_RMPl3Q4wRUuOok'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

Необходимо обучить модель используя Spark MLlib (модель на ваш выбор, например Decision Tree) и получить accuracy.
Подробнее тут: https://spark.apache.org/docs/latest/ml-classification-regression.html

In [ ]:
# код
from pyspark.ml import Pipeline

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
data = spark.read.format("libsvm").load("mnist-digits-train.txt")

In [ ]:
data_test = spark.read.format("libsvm").load("mnist-digits-test.txt")#вручную добавил в блокнот 779:0 780:0

In [ ]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

In [ ]:
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", handleInvalid='skip').fit(data)

In [ ]:
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [ ]:
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [ ]:
model = pipeline.fit(data)

In [ ]:
predictions = model.transform(data_test)

In [ ]:
predictions.select("prediction", "indexedLabel", "features").show(5)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(780,[202,203,204...|
|       6.0|         3.0|(780,[94,95,96,97...|
|       0.0|         0.0|(780,[128,129,130...|
|       5.0|         5.0|(780,[124,125,126...|
|       4.0|         8.0|(780,[150,151,159...|
+----------+------------+--------------------+
only showing top 5 rows



In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

In [ ]:
accuracy

0.6733708769107

In [ ]:
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.326629 


In [ ]:
id='1kUIrskM0zNH8u71G9M1BkHjRQYxvgAvh'
downloaded = drive.CreateFile({'id': id})
#downloaded.GetContentFile('data.zip')
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/regions.csv        
  inflating: data/departments.csv    
  inflating: data/jobs.csv           
  inflating: data/locations.csv      
  inflating: data/country.csv        
  inflating: data/employees.csv      
  inflating: data/job_history.csv    


Кто получает больше всего? Кто меньше всего?

In [ ]:
detector = UniversalDetector()
with open('data/regions.csv', 'rb') as fh:
    for line in fh:
        detector.feed(line)
        if detector.done:
            break
    detector.close()
print(detector.result)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [ ]:
ss.read.csv("data/regions.csv", sep='\t', encoding="ascii", header="True").createOrReplaceTempView("regions")

In [ ]:
jobser = ss.read.csv("data/jobs.csv", sep='\t', encoding="ascii", header="True").createOrReplaceTempView("jobs")

In [ ]:
# код
sq = spark.sql("""
SELECT * FROM jobs
""")

In [ ]:
sq.show()

+----------+--------------------+----------+----------+
|    JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|
+----------+--------------------+----------+----------+
|   AD_PRES|           President|     20080|     40000|
|     AD_VP|Administration Vi...|     15000|     30000|
|   AD_ASST|Administration As...|      3000|      6000|
|    FI_MGR|     Finance Manager|      8200|     16000|
|FI_ACCOUNT|          Accountant|      4200|      9000|
|    AC_MGR|  Accounting Manager|      8200|     16000|
|AC_ACCOUNT|   Public Accountant|      4200|      9000|
|    SA_MAN|       Sales Manager|     10000|     20080|
|    SA_REP|Sales Representative|      6000|     12008|
|    PU_MAN|  Purchasing Manager|      8000|     15000|
|  PU_CLERK|    Purchasing Clerk|      2500|      5500|
|    ST_MAN|       Stock Manager|      5500|      8500|
|  ST_CLERK|         Stock Clerk|      2008|      5000|
|  SH_CLERK|      Shipping Clerk|      2500|      5500|
|   IT_PROG|          Programmer|      4000|    

In [ ]:
sq.printSchema()

root
 |-- JOB_ID: string (nullable = true)
 |-- JOB_TITLE: string (nullable = true)
 |-- MIN_SALARY: string (nullable = true)
 |-- MAX_SALARY: string (nullable = true)



In [ ]:
from pyspark.sql.types import IntegerType, LongType, FloatType, DateType

In [ ]:
jobs_2 = sq.withColumn("MAX_SALARY_", sq.MAX_SALARY.cast(IntegerType()))\
.drop("MAX_SALARY")\
.withColumnRenamed("MAX_SALARY_", "MAX_SALARY")

In [ ]:
maxsi = jobs_2.select(max('MAX_SALARY'))

In [ ]:
jobs_2.select('*').where(col('MAX_SALARY') == maxsi.collect()[0][0]).show()

+-------+---------+----------+----------+
| JOB_ID|JOB_TITLE|MIN_SALARY|MAX_SALARY|
+-------+---------+----------+----------+
|AD_PRES|President|     20080|     40000|
+-------+---------+----------+----------+



In [ ]:
jobs_2.select('*').where(col('MAX_SALARY') == jobs_2.select(min('MAX_SALARY')).collect()[0][0]).show()

+--------+-----------+----------+----------+
|  JOB_ID|  JOB_TITLE|MIN_SALARY|MAX_SALARY|
+--------+-----------+----------+----------+
|ST_CLERK|Stock Clerk|      2008|      5000|
+--------+-----------+----------+----------+



Выведете топ 5 по зарплате.

In [ ]:
# код
jobs_2.select('*').sort(col('MAX_SALARY').desc()).show(5)

+-------+--------------------+----------+----------+
| JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|
+-------+--------------------+----------+----------+
|AD_PRES|           President|     20080|     40000|
|  AD_VP|Administration Vi...|     15000|     30000|
| SA_MAN|       Sales Manager|     10000|     20080|
| AC_MGR|  Accounting Manager|      8200|     16000|
| FI_MGR|     Finance Manager|      8200|     16000|
+-------+--------------------+----------+----------+
only showing top 5 rows



Сколько всего регионов? Сколько работников в каждом регионе?

In [ ]:
# код
skl = spark.sql("""
SELECT * FROM regions
""") 

In [ ]:
skl.show()#4 региона

+---------+--------------------+
|REGION_ID|         REGION_NAME|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
+---------+--------------------+



In [ ]:
locater = ss.read.csv("data/locations.csv", sep='\t', encoding="ascii", header="True").createOrReplaceTempView("locations")

In [ ]:
country = ss.read.csv("data/country.csv", sep='\t', encoding="ascii", header="True").createOrReplaceTempView("country")

In [ ]:
depart = ss.read.csv("data/departments.csv", sep='\t', encoding="ascii", header="True").createOrReplaceTempView("departments")

In [ ]:
empoy = ss.read.csv("data/employees.csv", sep='\t', encoding="ascii", header="True").createOrReplaceTempView("employees")

In [ ]:
jo = ss.read.csv("data/job_history.csv", sep='\t', encoding="ascii", header="True").createOrReplaceTempView("job_history")

In [ ]:
spark.sql("""
SELECT * FROM country
""").show()

+----------+------------+---------+
|COUNTRY_ID|COUNTRY_NAME|REGION_ID|
+----------+------------+---------+
|        AR|   Argentina|        2|
|        AU|   Australia|        3|
|        BE|     Belgium|        1|
|        BR|      Brazil|        2|
|        CA|      Canada|        2|
|        CH| Switzerland|        1|
|        CN|       China|        3|
|        DE|     Germany|        1|
|        DK|     Denmark|        1|
|        EG|       Egypt|        4|
|        FR|      France|        1|
|        IL|      Israel|        4|
|        IN|       India|        3|
|        IT|       Italy|        1|
|        JP|       Japan|        3|
|        KW|      Kuwait|        4|
|        ML|    Malaysia|        3|
|        MX|      Mexico|        2|
|        NG|     Nigeria|        4|
|        NL| Netherlands|        1|
+----------+------------+---------+
only showing top 20 rows



In [ ]:
spark.sql("""
SELECT * FROM locations
""").show(2)

+-----------+--------------------+-----------+------+--------------+----------+
|LOCATION_ID|      STREET_ADDRESS|POSTAL_CODE|  CITY|STATE_PROVINCE|COUNTRY_ID|
+-----------+--------------------+-----------+------+--------------+----------+
|       1000|1297 Via Cola di Rie|      00989|  Roma|          null|        IT|
|       1100|93091 Calle della...|      10934|Venice|          null|        IT|
+-----------+--------------------+-----------+------+--------------+----------+
only showing top 2 rows



In [ ]:
spark.sql("""
SELECT * FROM departments
""").show(2)

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
|           20|      Marketing|       201|       1800|
+-------------+---------------+----------+-----------+
only showing top 2 rows



In [ ]:
spark.sql("""
SELECT * FROM employees
""").show(2)

+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567| 17.06.03|AD_PRES| 24000|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568| 21.09.05|  AD_VP| 17000|          null|       100|           90|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
only showing top 2 rows



In [ ]:
skl_ = spark.sql("""
SELECT count(EMPLOYEE_ID), REGION_ID FROM employees
  RIGHT JOIN departments ON employees.DEPARTMENT_ID = departments.DEPARTMENT_ID
    RIGHT JOIN locations ON locations.LOCATION_ID = departments.LOCATION_ID
      RIGHT JOIN country ON country.COUNTRY_ID = locations.COUNTRY_ID
        GROUP BY REGION_ID
          ORDER BY REGION_ID
""").show()

+------------------+---------+
|count(EMPLOYEE_ID)|REGION_ID|
+------------------+---------+
|                36|        1|
|                70|        2|
|                 0|        3|
|                 0|        4|
+------------------+---------+



In [ ]:
spark.sql("""
SELECT * FROM jobs
""") .show(2)

+-------+--------------------+----------+----------+
| JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|
+-------+--------------------+----------+----------+
|AD_PRES|           President|     20080|     40000|
|  AD_VP|Administration Vi...|     15000|     30000|
+-------+--------------------+----------+----------+
only showing top 2 rows



Выведете всех работников из Китая.

In [ ]:
# код
spark.sql("""
SELECT * FROM employees
  FULL JOIN departments ON employees.DEPARTMENT_ID = departments.DEPARTMENT_ID
    FULL JOIN locations ON locations.LOCATION_ID = departments.LOCATION_ID
      FULL JOIN country ON country.COUNTRY_ID = locations.COUNTRY_ID
        WHERE locations.COUNTRY_ID = 'CN'
""").show()

#WHERE locations.COUNTRY_ID = 'CN'
#         ORDER BY locations.COUNTRY_ID

+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+-------------+---------------+----------+-----------+-----------+-------------------+-----------+-------+--------------+----------+----------+------------+---------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|EMAIL|PHONE_NUMBER|HIRE_DATE|JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|LOCATION_ID|     STREET_ADDRESS|POSTAL_CODE|   CITY|STATE_PROVINCE|COUNTRY_ID|COUNTRY_ID|COUNTRY_NAME|REGION_ID|
+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+-------------+---------------+----------+-----------+-----------+-------------------+-----------+-------+--------------+----------+----------+------------+---------+
|       null|      null|     null| null|        null|     null|  null|  null|          null|      null|         null|         null|           null|    

Укажите самую высокооплачиваемою должность.

In [ ]:
# код
full_table = spark.sql("""
SELECT * FROM jobs
  INNER JOIN employees ON employees.JOB_ID = jobs.JOB_ID
    INNER JOIN departments ON employees.DEPARTMENT_ID = departments.DEPARTMENT_ID
      INNER JOIN locations ON locations.LOCATION_ID = departments.LOCATION_ID
        INNER JOIN country ON country.COUNTRY_ID = locations.COUNTRY_ID
          ORDER BY SALARY 
""")

In [ ]:
full_table.show()

+--------+--------------------+----------+----------+-----------+----------+---------+--------+------------------+---------+--------+------+--------------+----------+-------------+-------------+----------------+----------+-----------+-----------+--------------------+-----------+-------------------+--------------+----------+----------+--------------------+---------+
|  JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|      PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|DEPARTMENT_ID| DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|LOCATION_ID|      STREET_ADDRESS|POSTAL_CODE|               CITY|STATE_PROVINCE|COUNTRY_ID|COUNTRY_ID|        COUNTRY_NAME|REGION_ID|
+--------+--------------------+----------+----------+-----------+----------+---------+--------+------------------+---------+--------+------+--------------+----------+-------------+-------------+----------------+----------+-----------+-----------+------------------

In [ ]:
jobs_3 = full_table.withColumn("SALARY_", full_table.SALARY.cast(IntegerType()))\
.drop("SALARY")\
.withColumnRenamed("SALARY_", "SALARY")

In [ ]:
jobs_3.select('JOB_TITLE', 'SALARY').sort(col('SALARY').desc()).show(1)

+---------+------+
|JOB_TITLE|SALARY|
+---------+------+
|President| 24000|
+---------+------+
only showing top 1 row



Выведете всех работников связанных с ИТ. Выведете их менеджеров. 

In [ ]:
# код не совсем понял, как определить связанных с IT
spark.sql("""
SELECT * FROM departments
WHERE DEPARTMENT_NAME = 'IT'
""").show()

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           60|             IT|       103|       1400|
+-------------+---------------+----------+-----------+



In [ ]:
spark.sql("""
SELECT FIRST_NAME, LAST_NAME, employees.MANAGER_ID FROM employees
  INNER JOIN departments ON employees.DEPARTMENT_ID = departments.DEPARTMENT_ID
    WHERE DEPARTMENT_NAME = 'IT'
""").show()

+----------+---------+----------+
|FIRST_NAME|LAST_NAME|MANAGER_ID|
+----------+---------+----------+
| Alexander|   Hunold|       102|
|     Bruce|    Ernst|       103|
|     David|   Austin|       103|
|     Valli|Pataballa|       103|
|     Diana|  Lorentz|       103|
+----------+---------+----------+



Выведете имя и фамилию работника, его текущую и предыдущую должности и сколько полных недель и дней прошло с момент изменения.


In [ ]:
# код
table_name = spark.sql("""
SELECT FIRST_NAME, LAST_NAME, employees.JOB_ID as JOB_ID_NOW, job_history.JOB_ID, START_DATE, END_DATE, HIRE_DATE FROM job_history
    RIGHT JOIN employees ON employees.EMPLOYEE_ID = job_history.EMPLOYEE_ID 
""")

In [ ]:
table_name.show()

+-----------+---------+----------+----------+----------+--------+---------+
| FIRST_NAME|LAST_NAME|JOB_ID_NOW|    JOB_ID|START_DATE|END_DATE|HIRE_DATE|
+-----------+---------+----------+----------+----------+--------+---------+
|     Steven|     King|   AD_PRES|      null|      null|    null| 17.06.03|
|      Neena|  Kochhar|     AD_VP|    AC_MGR|  28.10.01|15.03.05| 21.09.05|
|      Neena|  Kochhar|     AD_VP|AC_ACCOUNT|  21.09.97|27.10.01| 21.09.05|
|        Lex|  De Haan|     AD_VP|   IT_PROG|  13.01.01|24.07.06| 13.01.01|
|  Alexander|   Hunold|   IT_PROG|      null|      null|    null| 03.01.06|
|      Bruce|    Ernst|   IT_PROG|      null|      null|    null| 21.05.07|
|      David|   Austin|   IT_PROG|      null|      null|    null| 25.06.05|
|      Valli|Pataballa|   IT_PROG|      null|      null|    null| 05.02.06|
|      Diana|  Lorentz|   IT_PROG|      null|      null|    null| 07.02.07|
|      Nancy|Greenberg|    FI_MGR|      null|      null|    null| 17.08.02|
|     Daniel

In [ ]:
table_name.schema

StructType(List(StructField(FIRST_NAME,StringType,true),StructField(LAST_NAME,StringType,true),StructField(JOB_ID_NOW,StringType,true),StructField(JOB_ID,StringType,true),StructField(START_DATE,StringType,true),StructField(END_DATE,StringType,true),StructField(HIRE_DATE,StringType,true)))

In [ ]:
table_name = table_name.withColumn("START_DATE_", table_name.START_DATE.cast(DateType()))\
.drop("START_DATE")\
.withColumnRenamed("START_DATE_", "START_DATE")

In [ ]:
table_name.show(4)#печальные nullsы

+----------+---------+----------+----------+----------+--------+---------+
|FIRST_NAME|LAST_NAME|JOB_ID_NOW|    JOB_ID|START_DATE|END_DATE|HIRE_DATE|
+----------+---------+----------+----------+----------+--------+---------+
|    Steven|     King|   AD_PRES|      null|      null|    null|     null|
|     Neena|  Kochhar|     AD_VP|    AC_MGR|  28.10.01|15.03.05|     null|
|     Neena|  Kochhar|     AD_VP|AC_ACCOUNT|  21.09.97|27.10.01|     null|
|       Lex|  De Haan|     AD_VP|   IT_PROG|  13.01.01|24.07.06|     null|
+----------+---------+----------+----------+----------+--------+---------+
only showing top 4 rows



Выведете уникальные телефонные номера

In [ ]:
spark.sql("""
SELECT DISTINCT PHONE_NUMBER FROM employees
""").show(2)

+------------------+
|      PHONE_NUMBER|
+------------------+
|011.44.1344.429018|
|      515.127.4566|
+------------------+
only showing top 2 rows



In [ ]:
# код
phone_table = spark.sql("""
SELECT PHONE_NUMBER FROM employees
""")

In [ ]:
phone_table.select('*').distinct().drop_duplicates().show()

+------------------+
|      PHONE_NUMBER|
+------------------+
|011.44.1344.429018|
|      515.127.4566|
|      515.127.4564|
|011.44.1344.429278|
|      515.123.4569|
|      650.124.1434|
|      650.123.2234|
|011.44.1344.498718|
|      650.127.1634|
|      515.127.4561|
|011.44.1345.629268|
|      515.127.4562|
|011.44.1644.429264|
|011.44.1644.429262|
|      650.501.1876|
|      650.127.1834|
|011.44.1343.529268|
|011.44.1644.429265|
|      515.123.8181|
|      650.507.9833|
+------------------+
only showing top 20 rows



Есть ли сотрудники с одинаковыми фамилиями и сколько их.

In [ ]:
# код
spark.sql("""
SELECT LAST_NAME, count(LAST_NAME) FROM employees
GROUP BY LAST_NAME HAVING count(LAST_NAME) > 1
""").show()

+---------+----------------+
|LAST_NAME|count(LAST_NAME)|
+---------+----------------+
|    Smith|               2|
|     King|               2|
|Cambrault|               2|
|   Taylor|               2|
+---------+----------------+

